# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Write functions to parse specific information from a web page
* Iterate over successive web pages in order to create a dataset

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [15]:
def retrieve_titles(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    return [h3.find('a').attrs['title'] for h3 in book_container.findAll('h3')]

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given html page and return a list of the star-ratings for the books. These star ratings should be formatted as integers.

In [16]:
import re
def retrieve_ratings(soup):
    #Your code here
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    regex = re.compile("star-rating (.*)")
    book_container.findAll('p', {"class" : regex}) #Initial Trial in developing the script
    star_ratings = []
    for p in book_container.findAll('p', {"class" : regex}):
        star_ratings.append(p.attrs['class'][-1])
    star_dict = {'One': 1, 'Two': 2, 'Three':3, 'Four': 4, 'Five':5} #Manually create a dictionary to translate to numeric
    star_ratings = [star_dict[s] for s in star_ratings]
    return star_ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [17]:
def retrieve_prices(soup):
    #Your code here
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    prices = [p.text for p in book_container.findAll('p', class_="price_color")] #Keep cleaning it up
    return [float(p[1:]) for p in prices] #Removing the pound sign and converting to float


## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [18]:
def retrieve_availabilities(soup):
    warning = soup.find('div', class_="alert alert-warning")
    book_container = warning.nextSibling.nextSibling
    avails = book_container.findAll('p', class_="instock availability")
    return [a.text.strip() for a in book_container.findAll('p', class_="instock availability")] #Finalize the selection

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [19]:
#Your code here
import pandas as pd
import requests
from bs4 import BeautifulSoup
new_titles = []
new_star_ratings = []
new_prices = []
new_avails = []

df = pd.DataFrame()
for i in range(1,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles.extend(retrieve_titles(soup))
    new_star_ratings.extend(retrieve_ratings(soup))
    new_prices.extend(retrieve_prices(soup))
    new_avails.extend(retrieve_availabilities(soup))

df = pd.DataFrame([new_titles, new_star_ratings, new_prices, new_avails]).transpose()
df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
df

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used url hacking to generate each successive page url, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [20]:
#Your code here
def nextURL(soup):
    next1 = soup.find('li', class_="next")
    return next1.find('a').attrs['href']

In [33]:
new_titles = []
new_star_ratings = []
new_prices = []
new_avails = []

df = pd.DataFrame()
url = "http://books.toscrape.com/"
html_page = requests.get(url)
soup = BeautifulSoup(html_page.content, 'html.parser')
new_titles.extend(retrieve_titles(soup))
new_star_ratings.extend(retrieve_ratings(soup))
new_prices.extend(retrieve_prices(soup))
new_avails.extend(retrieve_availabilities(soup))


next2 = nextURL(soup)
url = "http://books.toscrape.com/" + next2

html_page = requests.get(url)
soup = BeautifulSoup(html_page.content, 'html.parser')

new_titles.extend(retrieve_titles(soup))
new_star_ratings.extend(retrieve_ratings(soup))
new_prices.extend(retrieve_prices(soup))
new_avails.extend(retrieve_availabilities(soup))

    
for i in range(3,51):
    next2 = nextURL(soup)
    url = "http://books.toscrape.com/catalogue/" + next2
    print(url)
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    print(i)
    new_titles.extend(retrieve_titles(soup))
    new_star_ratings.extend(retrieve_ratings(soup))
    new_prices.extend(retrieve_prices(soup))
    new_avails.extend(retrieve_availabilities(soup))

df = pd.DataFrame([new_titles, new_star_ratings, new_prices, new_avails]).transpose()
df.columns = ['Title', 'Star_Rating', 'Price_(pounds)', 'Availability']
df

http://books.toscrape.com/catalogue/page-3.html
3
http://books.toscrape.com/catalogue/page-4.html
4
http://books.toscrape.com/catalogue/page-5.html
5
http://books.toscrape.com/catalogue/page-6.html
6
http://books.toscrape.com/catalogue/page-7.html
7
http://books.toscrape.com/catalogue/page-8.html
8
http://books.toscrape.com/catalogue/page-9.html
9
http://books.toscrape.com/catalogue/page-10.html
10
http://books.toscrape.com/catalogue/page-11.html
11
http://books.toscrape.com/catalogue/page-12.html
12
http://books.toscrape.com/catalogue/page-13.html
13
http://books.toscrape.com/catalogue/page-14.html
14
http://books.toscrape.com/catalogue/page-15.html
15
http://books.toscrape.com/catalogue/page-16.html
16
http://books.toscrape.com/catalogue/page-17.html
17
http://books.toscrape.com/catalogue/page-18.html
18
http://books.toscrape.com/catalogue/page-19.html
19
http://books.toscrape.com/catalogue/page-20.html
20
http://books.toscrape.com/catalogue/page-21.html
21
http://books.toscrape.com/

,Title,Star_Rating,Price_(pounds),Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock
5,The Requiem Red,1,22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,4,33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,3,17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,4,22.6,In stock
9,The Black Maria,1,52.15,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!